In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import tensorflow as tf
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

# Initialize COCO

In [18]:
# define the path to the annotation file corresponding to the images you want to work with
baseDir='/Users/kyle/Repositories/coco'

trainAnnDir='person_keypoints_train2014'
valAnnDir='person_keypoints_val2014'
testAnnDir='image_info_test_dev2015'

trainDir='train2014'
valDir='val2014'
testDir='dev2015'

trainAnnotations='{}/annotations/{}.json'.format(baseDir,trainAnnDir)
valAnnotations='{}/annotations/{}.json'.format(baseDir,valAnnDir)
testAnnotations='{}/annotations/{}.json'.format(baseDir,testAnnDir)

trainData='{}/images/{}'.format(baseDir,trainDir)
valData='{}/images/{}'.format(baseDir,valDir)
testData='{}/images/{}'.format(baseDir,testDir)

print(trainAnnotations)
print(valAnnotations)
print(testAnnotations)
print(trainData)
print(valData)
print(testData)

# initialize a coco object
coco = COCO(trainAnnotations)

/Users/kyle/Repositories/coco/annotations/person_keypoints_train2014.json
/Users/kyle/Repositories/coco/annotations/person_keypoints_val2014.json
/Users/kyle/Repositories/coco/annotations/image_info_test_dev2015.json
/Users/kyle/Repositories/coco/images/train2014
/Users/kyle/Repositories/coco/images/val2014
/Users/kyle/Repositories/coco/images/dev2015
loading annotations into memory...
Done (t=11.23s)
creating index...
index created!


# Extract the appropriate images

** Now that we have downloaded the labels and initialialized a COCO object, we can examine the organizational structure of the dataset and specify specific categories of images to download  ** 

In [3]:
# get all images containing the 'person' category
catIds = coco.getCatIds(catNms=['person'])
imgIds = coco.getImgIds(catIds=catIds)

# Prepare training batch

In [41]:
batch_size = 10

batch_IDs = random.sample(imgIds, batch_size)
filenames = tf.constant(['{}/COCO_train2014_{:0>12}.jpg'.format(trainData,ID) for ID in batch_IDs])

annotations = [coco.loadAnns(coco.getAnnIds(imgIds=ID, catIds=catIds, iscrowd=None)) for ID in batch_IDs] 


In [42]:
# Fill lists with batch 

precision = tf.float16

bbox_list = [tf.stack([tf.constant(
                ann['bbox'],dtype=precision
            ) for ann in anns], axis=0) for anns in annotations]
keypoints_list = [tf.stack([tf.constant(
                np.reshape(ann['keypoints'],(-1,3)), dtype=precision
            ) for ann in anns], axis=0) for anns in annotations]
crowd_list = [tf.stack([tf.constant(
                [ann['iscrowd']]
            ) for ann in anns],axis=0) for anns in annotations]
area_list = [tf.stack([tf.constant(
                [ann['area']],dtype=precision
            ) for ann in anns], axis=0) for anns in annotations]
mask_list = [tf.stack([tf.constant(
                coco.annToMask(ann), dtype=tf.bool
            ) for ann in anns], axis = 0) for anns in annotations]

In [44]:
mask_list

[<tf.Tensor 'stack_607:0' shape=(3, 305, 432) dtype=bool>,
 <tf.Tensor 'stack_608:0' shape=(13, 640, 427) dtype=bool>,
 <tf.Tensor 'stack_609:0' shape=(2, 427, 640) dtype=bool>,
 <tf.Tensor 'stack_610:0' shape=(1, 640, 427) dtype=bool>,
 <tf.Tensor 'stack_611:0' shape=(8, 640, 480) dtype=bool>,
 <tf.Tensor 'stack_612:0' shape=(2, 480, 640) dtype=bool>,
 <tf.Tensor 'stack_613:0' shape=(1, 640, 427) dtype=bool>,
 <tf.Tensor 'stack_614:0' shape=(1, 640, 480) dtype=bool>,
 <tf.Tensor 'stack_615:0' shape=(5, 360, 640) dtype=bool>,
 <tf.Tensor 'stack_616:0' shape=(13, 480, 640) dtype=bool>]

In [49]:
bbox = tf.placeholder(tf.float32)
keypoints = tf.placeholder(tf.float32)
area = tf.placeholder(tf.float32)
mask = tf.placeholder(tf.float32)
iscrowd = tf.placeholder(tf.float32)

In [51]:
sess = tf.Session()
ann = annotations[0][0]

sess.run([bbox, keypoints, area, mask, iscrowd], 
         {bbox: ann['bbox'], keypoints: ann['keypoints'], area: ann['area'], mask: ann['segmentation'], iscrowd: ann['iscrowd']})

[array([ 120.48000336,  136.80999756,  161.25      ,  119.45999908], dtype=float32),
 array([ 255.,  154.,    2.,    0.,    0.,    0.,  251.,  151.,    2.,
           0.,    0.,    0.,  243.,  152.,    2.,  241.,  164.,    2.,
         238.,  164.,    2.,    0.,    0.,    0.,  242.,  174.,    2.,
           0.,    0.,    0.,  264.,  170.,    2.,  201.,  207.,    2.,
         200.,  205.,    2.,  173.,  221.,    2.,  172.,  218.,    2.,
         133.,  240.,    2.,  133.,  236.,    2.], dtype=float32),
 array(3410.22119140625, dtype=float32),
 array([[ 243.11999512,  139.08000183,  254.02000427,  141.80000305,
          257.20001221,  155.42999268,  251.75      ,  167.24000549,
          247.66000366,  171.33000183,  263.10998535,  167.24000549,
          281.73001099,  176.32000732,  270.38000488,  189.5       ,
          264.47000122,  174.50999451,  241.75999451,  177.69000244,
          211.33000183,  210.8500061 ,  189.52000427,  219.92999268,
          166.80999756,  226.28999329,

In [50]:
annotations[0][0]

{'area': 3410.22125,
 'bbox': [120.48, 136.81, 161.25, 119.46],
 'category_id': 1,
 'id': 421588,
 'image_id': 261172,
 'iscrowd': 0,
 'keypoints': [255,
  154,
  2,
  0,
  0,
  0,
  251,
  151,
  2,
  0,
  0,
  0,
  243,
  152,
  2,
  241,
  164,
  2,
  238,
  164,
  2,
  0,
  0,
  0,
  242,
  174,
  2,
  0,
  0,
  0,
  264,
  170,
  2,
  201,
  207,
  2,
  200,
  205,
  2,
  173,
  221,
  2,
  172,
  218,
  2,
  133,
  240,
  2,
  133,
  236,
  2],
 'num_keypoints': 13,
 'segmentation': [[243.12,
   139.08,
   254.02,
   141.8,
   257.2,
   155.43,
   251.75,
   167.24,
   247.66,
   171.33,
   263.11,
   167.24,
   281.73,
   176.32,
   270.38,
   189.5,
   264.47,
   174.51,
   241.76,
   177.69,
   211.33,
   210.85,
   189.52,
   219.93,
   166.81,
   226.29,
   148.64,
   234.92,
   131.84,
   256.27,
   120.48,
   244.46,
   122.75,
   235.37,
   148.19,
   219.93,
   175.44,
   204.94,
   176.8,
   200.85,
   185.89,
   199.04,
   210.87,
   188.13,
   235.85,
   152.7,
   239